<a href="https://colab.research.google.com/github/rdurbano/Machine-Learning/blob/master/Codenation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Importando as bibliotecas base**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# **Importando DataSet**

In [0]:
filePath = "/content/drive/My Drive/Codenation/train.csv"
df = pd.read_csv(filePath, delimiter=",", encoding="UTF8")

filePath = "/content/drive/My Drive/Codenation/test.csv"
dfTest = pd.read_csv(filePath, delimiter=",", encoding="UTF8")

colunasSelecionadas = list(dfTest.columns)
colunasSelecionadas.append('NU_NOTA_MT')

df = df[colunasSelecionadas]

dfresult = pd.DataFrame()
dfresult['NU_INSCRICAO'] = dfTest['NU_INSCRICAO'] 


# **Aplicando LABEL ENCODER e ONE HOT ENCODER**

Processo para transformar dados categóricos em numéricos

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# DATA CLEASING

for item in df.columns:
  if item[0:3] == "IN_":
    df[item] = df[item].astype(bool)

df['TP_ENSINO'] = df['TP_ENSINO'].astype(float)
df['TP_DEPENDENCIA_ADM_ESC'] = df['TP_DEPENDENCIA_ADM_ESC'].astype(float)
df['Q027'] = df['Q027'].fillna('Z')


features_list = ['NU_NOTA_CN','NU_NOTA_LC','NU_NOTA_CH','NU_NOTA_MT','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO']
def meanFill(data):
    for item in features_list:
        data[item] = data[item].fillna(data[item].mean())
    return data


df = df.fillna(0)

#ONE HOT ENCODER
enc = OneHotEncoder(handle_unknown='ignore')
labelencoder = LabelEncoder()

# for item in df.columns:
#   if item[0:2] == 'Q0' or item == 'SG_UF_RESIDENCIA':
#     enc_df = pd.DataFrame(enc.fit_transform(df[[item]]).toarray())
#     colunasRenomear = {}
#     for itemEncode in enc_df.columns:
#       colunasRenomear[itemEncode] = item +  str(itemEncode)
#     enc_df = enc_df.rename(columns=colunasRenomear)
#     df = df.join(enc_df)
 
# LABEL ENCODER
for item in df.columns:
  if item[0:2] == 'Q0' or item[0:3] == 'CO_' or item == 'SG_UF_RESIDENCIA' or item == 'TP_SEXO':
    df[item] = labelencoder.fit_transform(df[item])

# CONVERSOES
df['TP_SEXO'] = df['TP_SEXO'].astype(bool)

# DATA CLEASING
colDel = ['NU_INSCRICAO']
df.drop(colDel, axis = 1, inplace = True) # 'axis' = 0 (row) | axis = 1 (column)


# **Função para EDA**

In [0]:

def EDA_Codenation (dataSet):
  sns.set(style="whitegrid")

  # the histogram of the data
  fig, ax = plt.subplots(figsize= (16, 5))
  ax = sns.distplot(dataSet.NU_NOTA_MT,ax=ax)

  # Boxplot (Seaborn)
  plt.figure(figsize=(16, 5))
  ax = sns.boxplot(x=dataSet["NU_NOTA_MT"],width=20)

  #matriz de correlação
  plt.figure(figsize=(40, 30))
  corrMatrix = dataSet.corr()
  sns.heatmap(corrMatrix, annot=True)
  plt.show()


  #subplot
  fig, ax = plt.subplots(math.ceil(df.shape[1] /3 ), 3, figsize = (40, 50))
  fig.tight_layout()
  column =0
  row =0
  for item in dataSet.columns:
      sns.distplot(dataSet[item],ax = ax[row, column])
      column+=1
      if column == 3:
        column = 0
        row+=1


EDA_Codenation(df)

# **Feature Selection**

In [0]:
colManter = ['CO_UF_RESIDENCIA','SG_UF_RESIDENCIA','TP_SEXO','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO','Q001','Q002','Q006','Q024','Q025','Q047','NU_NOTA_MT']
#colManter = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO','NU_NOTA_MT']
df = df[colManter]

#EDA_Codenation(df)

## **Dataset de Treino - separando em Treino e Teste**


In [0]:
from sklearn.model_selection import train_test_split

# Separando PREDITORAS de TARGET
X_preditoras = df.drop('NU_NOTA_MT', axis=1, inplace=False)
Y_target = df['NU_NOTA_MT']

# Separando dados de TREINO e TESTE
Xtrain, Xtest, ytrain, ytest = train_test_split(X_preditoras, Y_target) 


# **Dataset de Teste - Aplicando data quality**
Utilizando o dataset de teste

In [0]:
# PREPARACAO DO DADO DE TESTE

#features_list = ['NU_NOTA_CN','NU_NOTA_LC','NU_NOTA_CH','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO']
#dfTest = meanFill(dfTest)
#dfTest = dfTest.fillna(dfTest.mean())
dfTest = dfTest.fillna(0)

colManter = ['CO_UF_RESIDENCIA','SG_UF_RESIDENCIA','TP_SEXO','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO','Q001','Q002','Q006','Q024','Q025','Q047']
dfTest = dfTest[colManter]

# LABEL ENCODER
for item in dfTest.columns:
  if item[0:2] == 'Q0' or item[0:3] == 'CO_' or item == 'SG_UF_RESIDENCIA' or item == 'TP_SEXO':
    dfTest[item] = labelencoder.fit_transform(dfTest[item])

# CONVERSOES
dfTest['TP_SEXO'] = dfTest['TP_SEXO'].astype(bool)

# **Normalizando utilizando MinMaxScaler**

In [0]:
# NORMALIZANDO


from sklearn.preprocessing import StandardScaler

min_max_scaler = preprocessing.MinMaxScaler()
Xtrain = min_max_scaler.fit_transform(Xtrain)  
Xtest = min_max_scaler.transform(dfTest.values)


# **Modelo de regressão Linear**

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate 

#instaciar modelo
kfold = KFold(n_splits=5, random_state=21, shuffle=True)
modelLR = LinearRegression()
modelLR.fit(Xtrain, ytrain)

# Cross Validation
resultado = cross_val_score(modelLR, Xtrain, ytrain, cv = kfold , scoring='neg_mean_squared_error')

# Print do resultado
msg = "%s | Erro médio: %f       Check: (%f)" % ('LinearRegression', resultado.mean(), resultado.std())
print(msg)


LinearRegression | Erro médio: -4497.044003       Check: (134.613693)


# **Modelo de Gradient Boosting**

The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

Two families of ensemble methods are usually distinguished:

In averaging methods, the driving principle is to build several estimators independently and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced.

Examples: Bagging methods, Forests of randomized trees, …

By contrast, in boosting methods, base estimators are built sequentially and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble.

Examples: AdaBoost, Gradient Tree Boosting, …

[link text](https://scikit-learn.org/stable/modules/ensemble.html)

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

# Criando o modelo
kfold = KFold(n_splits=5, random_state=21, shuffle=True)
modelGBR = GradientBoostingRegressor()
modelGBR.fit(Xtrain, ytrain)

# Cross Validation
resultado = cross_val_score(modelGBR, Xtrain, ytrain, cv = kfold , scoring='neg_mean_squared_error')

# Print do resultado
msg = "%s | Erro médio: %f       Check: (%f)" % ('LinearRegression', resultado.mean(), resultado.std())
print(msg)

LinearRegression | Erro médio: -4105.635497       Check: (131.036466)


GBM utilizando Pipeline e RandomizedSearchCV

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold,RandomizedSearchCV,cross_val_score

pipe_GBM = Pipeline([('scaler',  StandardScaler()),
            ('GradientBoostingRegressor', GradientBoostingRegressor())])

CV_pipe_GBM = RandomizedSearchCV(estimator = pipe_GBM, param_distributions = {},cv = 5,return_train_score=True, verbose=0)

CV_pipe_GBM.fit(X_preditoras, Y_target)

#predizendo em cima da base de teste
ypred = CV_pipe_GBM.predict(dfTest)

#depositando no dataset de resultado
dfresult['NU_NOTA_MT'] = np.around(ypred,2)

# **Modelo de Random Forest**

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Criando o modelo
kfold = KFold(n_splits=5, random_state=21, shuffle=True)
modelRFR = RandomForestRegressor()
modelRFR.fit(Xtrain, ytrain)

# Cross Validation
resultado = cross_val_score(modelRFR, Xtrain, ytrain, cv = kfold , scoring='neg_mean_squared_error')

# Print do resultado
msg = "%s | Erro médio: %f       Check: (%f)" % ('LinearRegression', resultado.mean(), resultado.std())
print(msg)

ypred = modelGBR.predict(Xtest)
dfresult['NU_NOTA_MT'] = np.around(ypred,2)

LinearRegression | Erro médio: -4372.788448       Check: (135.066998)


# **Exportação do DataSet para CSV**

In [0]:
dfresult.to_csv('codenationResposta.csv', index=False, header=True)